<a href="https://colab.research.google.com/github/jmelendezgeo/Exploratory-analysis-/blob/main/ImportExportColombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: This notebooks uses datasets saved on my repository. This data is available on source pages.

In this notebook we will work mainly with a historical record of imports and exports of Colombia and we will associate it with the respective products 

- Data sets load
- Cleaning, joining and preparation
- Data Visualization
- Storytelling and insights

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.io import output_notebook

# Data Load

This datasets are information on exports and imports of Colombia from 1962 to 2017. They were extracted from [OEC: The Observatory of Economic Complexity](https://oec.world/)

In [2]:
# COLOMBIA EXPORTS
url = 'https://raw.githubusercontent.com/jmelendezgeo/Data-Triathlon/main/colombia_exports.csv'
colombia_exports = pd.read_csv(url, sep = '|')
# COLOMBIA IMPORTS
url = 'https://raw.githubusercontent.com/jmelendezgeo/Data-Triathlon/main/colombia_imports.csv'
colombia_imports = pd.read_csv(url, sep = '|')
# COUNTRY NAMES
url = 'https://raw.githubusercontent.com/jmelendezgeo/Data-Triathlon/main/country_names.csv'
country_names = pd.read_csv(url, sep = '|')
# GROUP SITC
url = 'https://raw.githubusercontent.com/jmelendezgeo/Data-Triathlon/main/groups_sitc_rev2.csv'
groups_sitc = pd.read_csv(url, sep= '|')
# PRODUCTS SITC
url = 'https://raw.githubusercontent.com/jmelendezgeo/Data-Triathlon/main/products_sitc_rev2.csv'
products_sitc = pd.read_csv(url, sep = '|')



# Basic dataset info
**Exports**
```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389596 entries, 0 to 389595
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    389596 non-null  int64  
 1   Unnamed: 0.1  389596 non-null  int64  
 2   year          389596 non-null  int64  
 3   origin        389596 non-null  object 
 4   dest          389596 non-null  object 
 5   sitc4         389596 non-null  int64  
 6   export_val    389596 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 20.8+ MB
```
**Imports**

```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560492 entries, 0 to 560491
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    560492 non-null  int64  
 1   Unnamed: 0.1  560492 non-null  int64  
 2   year          560492 non-null  int64  
 3   origin        560492 non-null  object 
 4   dest          560492 non-null  object 
 5   sitc4         560492 non-null  int64  
 6   export_val    560492 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 29.9+ MB
```
**Products**
```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  988 non-null    int64 
 1   id          988 non-null    object
 2   sitc        988 non-null    int64 
 3   name        988 non-null    object
dtypes: int64(2), object(2)
memory usage: 31.0+ KB
```
**Groups**
```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  65 non-null     int64 
 1   id          65 non-null     int64 
 2   category    65 non-null     object
dtypes: int64(2), object(1)
memory usage: 1.6+ KB
```

**Countries**

```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  263 non-null    int64 
 1   id          263 non-null    object
 2   id_3char    263 non-null    object
 3   name        263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.3+ KB
```














# Labeling and combine transactions 

In [3]:
colombia_exports['Operation'] = 'export'
colombia_imports['Operation'] = 'import'

In [4]:
transactions = colombia_imports.append(colombia_exports)
transactions

,Unnamed: 0,Unnamed: 0.1,year,origin,dest,sitc4,export_val,Operation
0,0,11564,1962,civ,col,712,13000.0,import
1,2,17741,1962,cod,col,2925,2000.0,import
2,3,20216,1962,cog,col,2925,2000.0,import
3,6,29089,1962,egy,col,2631,40000.0,import
4,7,32443,1962,eth,col,2922,4000.0,import
...,...,...,...,...,...,...,...,...
389591,799400,124876487,2017,col,ury,6635,26526.5,export
389592,799402,124876489,2017,col,ury,8924,4285.5,export
389593,799403,124876490,2017,col,ury,8928,25667.5,export
389594,799404,124876491,2017,col,ury,8972,5306.0,export


In [5]:
# export_val indicates the USD amount in USD dollar of transactions. 
# Imports have Colombia as their destination, Exports have Colombia as their origin
# We are going to drop unnecessary columns

columns_to_keep = ['year','origin','dest','sitc4','export_val','Operation']
transactions = transactions[columns_to_keep]

# Add Country names and Continent fields

In [6]:
def create_continent(acr):
  """This function receives the acronym of a country 
  and relates the first two letters with the respective continents. 
  Returns the name of the continent  """

  if acr[0:2] == 'af':
    return 'Africa'
  elif acr[0:2] == 'as':
    return 'Asia'
  elif acr[0:2] == 'eu':
    return 'Europe'
  elif acr[0:2] == 'na':
    return 'North America'
  elif acr[0:2] == 'oc':
    return 'Oceania'
  elif acr[0:2] == 'sa':
    return 'South America'
  else:
    return 'Other'

In [7]:
country_names = country_names[['id','id_3char','name']]

#Origin name
transactions = (pd.merge(transactions,country_names, how = 'inner', left_on = 'origin', right_on = 'id_3char')
                .drop(columns='id_3char')
                .rename(columns={'name':'origin name'}))

# Destionation name

transactions = (pd.merge(transactions,country_names, how = 'inner', left_on = 'dest', right_on = 'id_3char')
                .drop(columns='id_3char')
                .rename(columns={'name':'destination name','sitc4':'sitc'}))


In [8]:
#@title Add Continents
transactions['origin continent'] = (transactions['id_x'].apply(lambda x : create_continent(x)))                                
transactions['destiny continent'] = (transactions['id_y'].apply(lambda x : create_continent(x)))
transactions.drop(columns=['id_x','id_y'],inplace = True)                                

# Add Product name and product group

In [9]:
#@title Product name
products_sitc = products_sitc[['id','sitc','name']]
transactions = (pd.merge(transactions,products_sitc[['sitc','name']], how='inner', on = 'sitc' )
                .rename(columns={'name':'product name'}))


In [10]:
#@title Group name
groups_sitc = groups_sitc[['id','category']]
# Now we need all characters in sitc
groups_sitc['id'] = groups_sitc['id'].astype(str).str.zfill(2) # p.e 7 is 07
transactions['sitc'] = transactions['sitc'].astype(str).str.zfill(4) # p.e  123 is 0123
transactions['sitc']=transactions['sitc'].str.extract(r'(^\d{2})') # The first 2 characters are the group sitc code
transactions = (pd.merge(transactions,groups_sitc,how='inner',left_on='sitc',right_on='id')
                .drop(columns=['sitc','id']))

In [11]:
transactions

,year,origin,dest,export_val,Operation,origin name,destination name,destiny continent,origin continent,product name,category
0,1962,civ,col,13000.0,import,Cote d'Ivoire,Colombia,South America,Africa,Coffee Extracts,"Coffee, tea, cocoa, spices, and manufactures t..."
1,1996,zaf,col,3175.0,import,South Africa,Colombia,South America,Africa,Coffee Extracts,"Coffee, tea, cocoa, spices, and manufactures t..."
2,1997,zaf,col,5685.0,import,South Africa,Colombia,South America,Africa,Coffee Extracts,"Coffee, tea, cocoa, spices, and manufactures t..."
3,1999,chn,col,1056.0,import,China,Colombia,South America,Asia,Coffee Extracts,"Coffee, tea, cocoa, spices, and manufactures t..."
4,2011,chn,col,49400.0,import,China,Colombia,South America,Asia,Coffee Extracts,"Coffee, tea, cocoa, spices, and manufactures t..."
...,...,...,...,...,...,...,...,...,...,...,...
938949,2011,col,ven,30104509.0,export,Colombia,Venezuela,South America,South America,Electric Current,Electric current
938950,2012,col,ven,55866310.0,export,Colombia,Venezuela,South America,South America,Electric Current,Electric current
938951,2013,col,ven,37267653.0,export,Colombia,Venezuela,South America,South America,Electric Current,Electric current
938952,2014,col,ven,102327794.0,export,Colombia,Venezuela,South America,South America,Electric Current,Electric current
